In [ ]:
!pip install pandas sqlalchemy



In [2]:
# Replace with the raw URL of your file on GitHub
github_file_url = 'https://raw.githubusercontent.com/g24ai1090/abc_retail_capstone/main/data/cleaned/online_retail_cleaned.csv'

# Download the file
!wget {github_file_url}

# Get the filename from the URL to use in pd.read_csv
import os
file_name = os.path.basename(github_file_url)

print(f"Downloaded {file_name}")

--2025-06-05 18:44:54--  https://raw.githubusercontent.com/g24ai1090/abc_retail_capstone/main/data/cleaned/online_retail_cleaned.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55976358 (53M) [text/plain]
Saving to: ‘online_retail_cleaned.csv’

online_retail_clean 100%[===================>]  53.38M   188MB/s    in 0.3s    

2025-06-05 18:44:56 (188 MB/s) - ‘online_retail_cleaned.csv’ saved [55976358/55976358]

Downloaded online_retail_cleaned.csv


In [3]:
import pandas as pd
import sqlite3

# Use the downloaded file name
df = pd.read_csv("online_retail_cleaned.csv")

conn = sqlite3.connect("retail_data.db")

df.to_sql("sales", conn, if_exists="replace", index=False)

conn.close()

print(" Data Ingested into 'Sales ' table sucessfully! ")

<ipython-input-3-9c277b10a7f3>:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("online_retail_cleaned.csv")


 Data Ingested into 'Sales ' table sucessfully! 


In [4]:
#Automate Periodic Ingestion (Simulate)

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Month'] = df['InvoiceDate'].dt.to_period('M')

months =sorted(df['Month'].unique())

for m in months:
    batch = df[df['Month'] == m].copy() # Create a copy to avoid SettingWithCopyWarning
    batch['Month'] = batch['Month'].astype(str)  # Convert Period to string

    conn = sqlite3.connect("retail_data.db")

    # Drop the table if it exists and recreate it with the correct schema
    conn.execute("DROP TABLE IF EXISTS sales;")
    batch.to_sql("sales", conn, if_exists="replace", index=False)

    conn.close()



In [5]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!pip install --upgrade pandas-gbq google-cloud-bigquery


In [6]:
import pandas as pd
df = pd.read_csv("online_retail_cleaned.csv")



<ipython-input-6-1155b9893915>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("online_retail_cleaned.csv")


In [7]:
from pandas_gbq import to_gbq

project_id = "capstone-project-462018"
dataset_table = "retail_data.db"

to_gbq(
    dataframe=df,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists='replace'  # or 'append' for incremental
)

print(f"✅ Data uploaded to BigQuery table: {dataset_table}")



/usr/local/lib/python3.11/dist-packages/pandas_gbq/schema/pandas_to_bigquery.py:157: UserWarning: Could not determine the type of columns: InvoiceNo
  warnings.warn(msg)
100%|██████████| 1/1 [00:00<00:00, 2744.96it/s]

✅ Data uploaded to BigQuery table: retail_data.db
